# Loading Feature Counts and Experiment Metadata

According to Helen there's an issues with the shRNAs ESRRA-1 and m35 bam files. I am not entirely sure what the issue is but i as a safety precaution we are removing them from our downstream analysis. This means we need to remove these from our feature count matrix.

In [22]:
require(org.Hs.eg.db, quietly = TRUE)

fc = readRDS('../data/010_lfc/hisat2.E2V2.Rds')
experiment_definitions = read.csv("../NEMpipeline/data/E2V2_experiment_definitions.csv")

Warning message in file(file, "rt"):
“cannot open file '../NEMpipeline/data/E2V2_experiment_definitions.csv': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


## Cleaning Metadata

Extracting all of the duplicate bam files from the experiment metadata and removing them from both the metadata and the count matrix generated from step 010_lfc. 

In [ ]:
# cleaning metadata
clean_metadata = experiment_definitions[!experiment_definitions$shRNA %in% c('ESRRA-1', 'm35'), ]
id = as.character(clean_metadata$Bam.File)  
id = vapply(strsplit(id,"/"),"[",4, FUN.VALUE=character(1))
id = vapply(strsplit(id,"-"),"[",2, FUN.VALUE=character(1))

In [ ]:
# extracting bad_shRNAs bam files for shRNA ESSRA-1 and m35
bad_shRNAs = experiment_definitions[experiment_definitions$shRNA %in% c('ESRRA-1', 'm35'), ]$Bam.File
bad_shRNAs = gsub('/', '.', bad_shRNAs)
bad_shRNAs = gsub('-', '.', bad_shRNAs)
bad_shRNAs = paste0('X.mnt.scratcha.fmlab.cook01.E2V2.', bad_shRNAs)
bad_shRNAs = c("bam\\.1", bad_shRNAs)
dedup = grep(paste(bad_shRNAs, collapse="|"), colnames(fc$counts))

In [ ]:
# clean the shRNAs
clean_metadata$shRNA = gsub('-', '_', clean_metadata$shRNA)

## Cleaning Count Matrix

Since where here we might as well annotate and clean the count matrix. Here i am creating new column names for the matrix and filtering out genes that fail to read the minimium expression level.

In [ ]:
# creating a edgeR Count Matrix for downstream analysis
x = edgeR::DGEList(counts=fc$counts[,-dedup], genes=fc$annotation[,c("GeneID","Length")])

# checking all that i have the same number of ids as i do columns in my edgeR Counts Matrix
names = grep(paste(id, collapse = "|"), colnames(x$counts), value = T)
stopifnot(all.equal(names,colnames(x$counts)[1:439]))

# extracting samples using the correct names
x$counts = x$counts[,which(colnames(x$counts) %in% names)]
x$samples = x$samples[which(rownames(x$samples) %in% names),]

In [23]:
# Look up the gene names in the UCSC hg38 annotation
stopifnot(all.equal(as.integer(rownames(x$counts)),x$genes$GeneID))
rownames(x$counts) = x$genes$GeneID
rownames(x$counts) = annotate::lookUp(rownames(x$counts), 'org.Hs.eg', 'SYMBOL')

Warning message in all.equal(as.integer(rownames(x$counts)), x$genes$GeneID):
“NAs introduced by coercion”


ERROR: Error: as.integer(rownames(x$counts)) and x$genes$GeneID are not equal:
  'is.NA' value mismatch: 0 in current 17210 in target


In [24]:
# rename columns so they using the data more
samples_all = paste(clean_metadata$Pool, clean_metadata$Barcode, 
                 clean_metadata$Cell.line, clean_metadata$Gene, 
                 clean_metadata$shRNA, clean_metadata$Biological.Rep)

# don't overwrite samples from config
colnames(x$counts) = samples_all

In [26]:
expr.cutoff = 0.5

# filter out genes that don't vary by more than expr.cutoff in more than 2 experiments
keep = rowSums(edgeR::cpm(x) > expr.cutoff) >= 2
x = x[keep, , keep.lib.sizes=FALSE]

counts = x$counts

## Saving Count Matrix and Metadata

In [29]:
saveRDS(x, '../data/010_lfc/cleaned_feature_counts.Rds')
write.csv(clean_metadata, '../data/metadata/clean_metadata.csv')